# Gas Residential Q2 and YTD Transformation Script
<img src="images\img_1.png" alt="banner_1" size="100%">

1. The script `gas_res_transform.ipynb` takes files from `/data_summary` and creates two new Excel files in `/transformed_data`.
2. The new files will be named:
   - `gas_residential_2025_q2.xlsx`
   - `gas_residential_2025_q2_ytd.xlsx`
3. The original files in `/data_summary` contain Q2 and YTD data from JD Power's Gas Residential summaries.
4. Each new file in `transformed_data` will include around nine data transformations, or nine sheets.
5. For more details, check the `notes.docx` file.

## Import the libraries needed

In [85]:
import pandas as pd
import numpy as np

# Data checker coming soon
<img src="images\img_2.png" alt="banner_2" size="100%">

I'll add code later to check if the data summaries are set up correctly from Powersource. I'll also create a checklist too for manual corrections.

In [86]:
def validate_data(workbook):
    print("Coming soon!")

In [87]:
validate_data("workbook")

Coming soon!


# Repetitive tasks

### Renaming these dimensions for the transformed sheets
-	Ease - Ease of doing Business
-	Cost - Total Monthly Cost
-	Safety - Safety & Reliability
-	Resolving Problems - Problem Resolution
-	Info Provided - Information Provided

In [88]:
def replace_dimensions(df):
    replacements = {
    'ease': "Ease of doing business",
    'cost': "Total monthly cost",
    'safety': 'Safety and Reliability',
    'resolving problems': 'Problem Resolution',
    'info provided': 'Information Provided'
}
    df["Dimension"] = df["Dimension"].apply(lambda x: replacements.get(x.lower(), x))
    return df

### Concatenation tables or dataframes within a list if needed
Many times the first output from a transformation is a list of multiple dataframes. We often need to merge every two items on a list of dimensions, so I'm setting this up for any output that needs this function. 

In [89]:
def concat_df_rows(df_list, cols):
    if not isinstance(cols,list):
        raise TypeError("( •_•)🔍 cols variable needs to be a list.")
    merged_pairs = [pd.merge(df_list[i], df_list[i+1], how="inner", on=cols) for i in range(0, len(df_list), 2)]
    merged_pairs = pd.concat(merged_pairs, axis=0)
    return merged_pairs

### This `process_and_stack` function are for multiple sheets that need to go through one function.

In [90]:
def process_and_stack(df, names, function, dimensions):
    if not isinstance(dimensions, list):
        raise TypeError("( •_•)🔍 cols variable needs to be a list.")
    all_rows = []
    for name in names:
        transformed = function(df, name)
        all_rows.extend(transformed)
    final_df = concat_df_rows(all_rows,dimensions)
    return final_df

## Turn column values to percent

In [91]:
def turn_to_percent(df, col):
    df[col] = df[col].astype(float)
    df[col] = np.round(df[col])
    df[col] = df[col].astype(int)
    return df

# Pull Q2 and YTD data
We removed the first sheet by just setting a variable to a slice.

In [92]:
q_df = pd.read_excel(r"C:\Users\A00126508\OneDrive - PSEG\VOC Team - JD Power\Gas Residential\JD Power Gas Residential\data_summary\q2_data\JDP Gas Rez Q2 Powersource output 8-1-25 v2.xlsx", sheet_name=None)
ytd_df = pd.read_excel(r"C:\Users\A00126508\OneDrive - PSEG\VOC Team - JD Power\Gas Residential\JD Power Gas Residential\data_summary\ytd_data\JDP Gas Rez YTD Powersource output 8-1-25.xlsx", sheet_name=None)

In [93]:
q2_sheetnames = list(q_df.keys())[1:]
ytd_sheetnames = list(ytd_df.keys())[1:]

# Transform Gas Residential Dimension Index
This will create the `Toplines` sheet for Q2 and YTD data

### Sheet(s)

In [94]:
grdi = q2_sheetnames[0]

In [95]:
grdi_2 = ytd_sheetnames[0]

### Functions

In [96]:
def transform_gas_res_dim_index(df, name):
    rows = []

    data = df[name].loc[3:,"Unnamed: 2":]
    d_col = df[name].iloc[4:]["Table of Contents"]
    for col in data:
        temp_df = pd.DataFrame({
            "Dimension": d_col,
        })
        if data[col].iloc[0] == "Rank":
            temp_df[data[col].iloc[0]] = data[col][1:]
        else:
            label = data[col].iloc[0]
            temp_df["Brand Name"] = label
            temp_df["Mean"] = data[col][1:]
        rows.append(temp_df)
    return rows

### Get Q2 2025 and Q2 YTD 2025

In [97]:
q2_1 = transform_gas_res_dim_index(q_df, grdi)
ytd_1 = transform_gas_res_dim_index(ytd_df, grdi_2)

In [98]:
q2_1 = concat_df_rows(q2_1,["Dimension"])
ytd_1 = concat_df_rows(ytd_1,["Dimension"])

In [99]:
q2_1 = replace_dimensions(q2_1)

In [100]:
ytd_1 = replace_dimensions(ytd_1)

# Tranform KPI %s Met Not-Met all brands
We will transform this sheet into `KPI % All Brands` sheet in the transformed Q2 and YTD data

### Sheet(s)

In [101]:
q2_kpi = q2_sheetnames[1]

In [102]:
ytd_kpi = ytd_sheetnames[1]

### Functions

In [103]:
def transform_kpi_brands(df, name):
    rows = []
    o_col = df[name].loc[3:,"Unnamed: 2":]
    q_col = df[name].iloc[5:]["Table of Contents"].ffill()
    met = df[name].iloc[5:]["Unnamed: 1"] 
    for col in o_col:
        temp_df = pd.DataFrame({
            "Question": q_col,
            "Met": met
        })
        if o_col[col].iloc[1] == "%":
            temp_df["%"] = o_col[col].iloc[2:]
        else:
            temp_df["Rank"] = o_col[col].iloc[2:]
        rows.append(temp_df)
    return rows

In [104]:
# def turn_to_percent(df, col):
#     df[col] = df[col].astype(float)
#     df[col] = np.round(df[col])
#     df[col] = df[col].astype(int)
#     return df

### Get Q2 2025 and Q2 YTD 2025

In [105]:
q2_kpi_df = concat_df_rows(transform_kpi_brands(q_df,q2_kpi), ["Question", "Met"])

In [106]:
q2_kpi_df = turn_to_percent(q2_kpi_df,"%")

In [107]:
ytd_kpi_df = concat_df_rows(transform_kpi_brands(ytd_df,ytd_kpi),["Question","Met"])

In [108]:
ytd_kpi_df = turn_to_percent(ytd_kpi_df,"%")

# PSEG KPIs by Dimension Transformation
We will transform the `PSEG KPIs by Dimensions Index Met Not-Met and base size` into the `PSEG KPIs` sheet. We will also create a separate table containing the differences between met and not met.

### Sheet(s)

In [109]:
q2_pseg_kpi = q2_sheetnames[2]

In [110]:
ytd_pseg_kpi = ytd_sheetnames[2]

### Functions

In [111]:
def transform_pseg_kpi(df, name):
    rows = []
    q_col = df[name].iloc[4:]["Table of Contents"].ffill()
    met = df[name].iloc[4:]["Unnamed: 1"]
    o_col = df[name].loc[2:,"Unnamed: 2":]

    for col in o_col:
        temp_df = pd.DataFrame({
            "Question": q_col,
            "Met": met
        })
       
        if o_col[col].iloc[1] == "Mean":
            dim = o_col[col].iloc[0]
            temp_df["Mean"] = o_col[col].iloc[1:]
            temp_df["Dimension"] = dim
        else:
            temp_df["Unweighted base"] = o_col[col].iloc[1:]
            temp_df["Dimension"] = dim
        rows.append(temp_df)
    return rows

### Get Q2 2025 and Q2 YTD 2025

In [112]:
q2_dim_kpi = transform_pseg_kpi(q_df, q2_pseg_kpi)

In [113]:
q2_dim_kpi_df = concat_df_rows(q2_dim_kpi,["Question","Dimension","Met"])

In [114]:
q2_dim_kpi_df = replace_dimensions(q2_dim_kpi_df)

### Get Difference Table for Q2 and YTD

In [115]:
def get_difference(df):
    df1 = df.groupby(['Question', 'Dimension']).agg({
    'Mean': lambda x: x.iloc[1] - x.iloc[0]
}).reset_index()
    df1.rename(columns={"Mean":"Difference"}, inplace=True)
    return df1

In [116]:
q2_dim_diff = get_difference(q2_dim_kpi_df)

In [117]:
q2_dim_diff

,Question,Dimension,Difference
0,Aware of assistance programs,Digital channels,138.672381
1,Aware of assistance programs,Ease of doing business,164.821303
2,Aware of assistance programs,Information provided,128.958479
3,Aware of assistance programs,Overall Satisfaction,138.140546
4,Aware of assistance programs,People,132.841335
...,...,...,...
157,Utility helped to lower bill by showing how to...,People,161.484377
158,Utility helped to lower bill by showing how to...,Problem resolution,153.350670
159,Utility helped to lower bill by showing how to...,Safety and reliability,124.711630
160,Utility helped to lower bill by showing how to...,Total monthly cost,116.678192


In [118]:
ytd_dim_kpi_df = transform_pseg_kpi(ytd_df,ytd_pseg_kpi)
ytd_dim_kpi_df = concat_df_rows(ytd_dim_kpi_df,["Question","Dimension","Met"])

In [119]:
ytd_dim_diff = get_difference(ytd_dim_kpi_df)

In [120]:
ytd_dim_diff = replace_dimensions(ytd_dim_diff)

# Dimension KPIs %s & Rank
This transformation `Dimension KPIs %s & Rank` will be using these sheets as input
- Safety & Reliability KPIs %s & Rank Met Not-Met all brands
- ProblemResolution KPI %s & Rank Met Not-Met all brands
- Ease KI %s & Rank Met Not-Met all brands
- Digital Channels KPI %s & Rank Met Not-Met all brands
- People KPI %s & Rank Met Not-Met all brands
- Cost KPI %s & Rank Met Not-Met all brands
- Trust KPI %s & Rank Met Not-Met all brands
- Information Provided KPI %s & Rank Met Not-Met all brands


### Sheet(s)

In [121]:
q2_k_r = q2_sheetnames[3:11]

In [122]:
ytd_k_r = ytd_sheetnames[3:11]

### Functions

In [123]:
def transform_kpi_rank(df,name):
    rows = []

    q_col = df[name].iloc[5:]["Table of Contents"].ffill()
    met = df[name].iloc[5:]["Unnamed: 1"]
    o_col = df[name].loc[3:,"Unnamed: 2":]
    
    for col in o_col:
        temp_df = pd.DataFrame({
            "Question": q_col,
            "Met": met,
            "Dimension": name
        })
        if o_col[col].iloc[1] == "%":
            temp_df["%"] = o_col[col].iloc[1:]
            bn = o_col[col].iloc[0]
            temp_df["Brand Name"] = bn
        else:
            temp_df["Rank"] = o_col[col].iloc[1:]
            temp_df["Brand Name"] = bn
        rows.append(temp_df)
    return rows

### Get Q2 2025 and Q2 YTD 2025

In [124]:
q_kpi_r_all = process_and_stack(q_df, q2_k_r, transform_kpi_rank,["Question","Met","Brand Name","Dimension"])

In [125]:
q_kpi_r_all = turn_to_percent(q_kpi_r_all,"%")

In [126]:
q_kpi_r_all["Dimension"] = q_kpi_r_all["Dimension"].apply(lambda x: x.split('KPI')[0].strip())

In [127]:
q_kpi_r_all = replace_dimensions(q_kpi_r_all)

In [128]:
ytd_kpi_r_all = process_and_stack(ytd_df,ytd_k_r,transform_kpi_rank,["Question","Met","Brand Name","Dimension"])
ytd_kpi_r_all = turn_to_percent(ytd_kpi_r_all,"%")

In [129]:
ytd_kpi_r_all = replace_dimensions(ytd_kpi_r_all)

In [130]:
ytd_kpi_r_all["Dimension"] = ytd_kpi_r_all["Dimension"].apply(lambda x: x.split('KPI')[0].strip())

In [131]:
ytd_kpi_r_all = replace_dimensions(ytd_kpi_r_all)

# Demographics KPI Transformation
We will transform the `PSEG Demographics KPI` into the `Demographics KPI` sheet.

### Get Sheet(s)

In [132]:
q_demo_name = q2_sheetnames[11]
ytd_demo_name = ytd_sheetnames[11]

### Functions

In [133]:
def transform_demographics(df, name):
    rows = []

    d_col = df[name].iloc[4:]["Table of Contents"].ffill()
    s_col = df[name].iloc[4:]["Unnamed: 1"]
    o_col = df[name].loc[2:,"Unnamed: 2":]

    for col in o_col:
        temp_df = pd.DataFrame({
            "Demographic": d_col,
            "Segment": s_col
        })
        if o_col[col].iloc[1] == "Mean":
            temp_df["Mean"] = o_col[col].iloc[2:]
        else:
            temp_df["Unweighted base"] = o_col[col].iloc[2:]
        rows.append(temp_df)
    return rows

### Get Q2 2025 and Q2 YTD 2025

In [134]:
q_demo_df = concat_df_rows(transform_demographics(q_df, q_demo_name), ["Demographic", "Segment"])

In [135]:
ytd_demo_df = concat_df_rows(transform_demographics(ytd_df,ytd_demo_name), ["Demographic", "Segment"])

# County KPI Transformation
We will transform the `PSEG County KPIs` into the `County KPI` sheet.

### Sheet(s)

In [136]:
q_c_name = q2_sheetnames[12]
ytd_c_name = ytd_sheetnames[12]

### Functions

In [137]:
def transform_county(df, name):
    rows = []
    c_col = df[name].iloc[4:]["Unnamed: 1"]
    o_col = df[name].loc[2:,"Unnamed: 2":]
    for col in o_col:
        temp_df = pd.DataFrame({
            "County": c_col
        })
        if o_col[col].iloc[1] == "Mean":
            dim = o_col[col].iloc[0]
            temp_df["Mean"] = o_col[col].iloc[2:]
            temp_df["Dimension"] = dim
        else:
            temp_df["Unweighted base"] = o_col[col].iloc[2:]
            temp_df["Dimension"] = dim
        rows.append(temp_df)
    return rows


### Get Q2 2025 and Q2 YTD 2025

In [138]:
q_c_df = transform_county(q_df, q_c_name)
ytd_c_df = transform_county(ytd_df, ytd_c_name)

In [139]:
q_c_df = concat_df_rows(q_c_df, ["Dimension","County"])
ytd_c_df = concat_df_rows(ytd_c_df,["Dimension","County"])

In [140]:
q_c_df = replace_dimensions(q_c_df)
ytd_c_df = replace_dimensions(ytd_c_df)

# Tranform NPS
We will transform the `NPS Groups all brands` into the `NPS Score all brands` sheet.

### Get sheet(s)

In [141]:
q_nps_1 = q2_sheetnames[13]
q_nps_2 = q2_sheetnames[14]


In [142]:
ytd_nps_1 = ytd_sheetnames[13]
ytd_nps_2 = ytd_sheetnames[14]

### Functions

In [143]:
def transform_nps(df, n1, n2):
    rows = []

    o_col_1 = df[n1].loc[3:,"Table of Contents":]

    o_col_2 = df[n2].loc[3:,"Unnamed: 2":]
    l2 = o_col_2.iloc[1:]
    
    g_df = pd.DataFrame({})
    g_df[o_col_2.iloc[0]] =  l2


    for col in o_col_1:
        label = o_col_1[col].iloc[0]
        temp_df = pd.DataFrame({
            label: o_col_1[col].iloc[1:]
        })

        rows.append(temp_df)
    rows.append(g_df)
    del rows[0]
    del rows[1]
   
    combined = pd.concat(rows, axis=1)
    combined.columns = combined.columns.fillna("Brand Name")
    return combined
    

### Get Q2 2025 and Q2 YTD 2025

In [144]:
q_nps_df = transform_nps(q_df,q_nps_1,q_nps_2)
ytd_nps_df = transform_nps(ytd_df, ytd_nps_1, ytd_nps_2)

# Transform PSEG Dimension Index Scores
These sheets will be transformed into the `PSEG Dimension Index Scores` sheet:
- PSEG Cost Index Score Met Not-Met and base size
- PSEG Safety Index Score Met Not-Met and base size
- PSEG Problem Resolution Index Score Met Not-Met and base size
- PSEG Ease Index Score Met Not-Met and base size
- PSEG Digital Channels Index Score Met Not-Met and base size
- PSEG People Index Score Met Not-Met and base size
- PSEG Trust Index Score Met Not-Met and base size
- PSEG Info Provided Score Met Not-Met and base size


### Get Sheet(s)

In [145]:
q_idxs = q2_sheetnames[15:23]

In [146]:
ytd_idxs = ytd_sheetnames[15:23]

### Functions

In [147]:
def transform_dim_index(df, name):
    rows = []
    q_col = df[name].iloc[4:]["Table of Contents"].ffill()
    met = df[name].iloc[4:]["Unnamed: 1"]
    o_col = df[name].loc[2:,"Unnamed: 2":]
    label = o_col.iloc[0]["Unnamed: 2"]
    for col in o_col:
        temp_df = pd.DataFrame({
            "Question": q_col,
            o_col[col].iloc[1]: o_col[col].iloc[2:],
            "Dimension": label,
            "Met": met
        })
        rows.append(temp_df)
    return rows

### Get Q2 2025 and Q2 YTD 2025

In [148]:
q_idx_df = process_and_stack(q_df,q_idxs,transform_dim_index,["Question","Dimension","Met"])

In [149]:
q_idx_df = replace_dimensions(q_idx_df)

In [150]:
ytd_idx_df = process_and_stack(ytd_df, ytd_idxs, transform_dim_index,["Question","Dimension","Met"])

In [151]:
ytd_idx_df = replace_dimensions(ytd_idx_df)

# Transform All Historicals
| Section       | Transformation Description |
|---------------|----------------------------|
| East Large | `East Large Historical KPIs %s` will be transformed into `East Large Historical`. |
| PSEG       | `PSEG Historical %s Met Not Met` will be transformed into `PSEG Historical`. |
| Dimensions | These sheets will be transformed into `Dimension Historicals`. |


In [152]:
# East Large Historical Sheet(s)
elh_q_name = q2_sheetnames[23]
elh_ytd_name = ytd_sheetnames[23]

In [153]:
# PSEG Historical Sheet(s)
plh_q_name = q2_sheetnames[24]
plh_ytd_name = ytd_sheetnames[24]

In [154]:
# Dimensions Historical Sheet(s)
q_dim_names = q2_sheetnames[25:]
ytd_dim_names = ytd_sheetnames[25:]

### Functions

In [155]:
def transform_historicals(df, name):
    rows = []

    q_col = df[name].iloc[5:]["Table of Contents"].ffill()
    met = df[name].iloc[5:]["Unnamed: 1"]
    o_col = df[name].loc[3:,"Unnamed: 2":]

    for col in o_col:
        label = o_col[col].iloc[0]
        percent = o_col[col].iloc[1]
        temp_df = pd.DataFrame({
            "Question": q_col,
            "Met": met,
            "Quarter": label,
            percent: o_col[col].iloc[2:]
        })
        rows.append(temp_df)
        final_df = pd.concat(rows, axis=0)
    return final_df

### Get Q2 and YTD data for East Large Historical

In [156]:
q_e_df = transform_historicals(q_df, elh_q_name)
q_e_df["%"] = q_e_df["%"] * 100
q_e_df = turn_to_percent(q_e_df,"%")


In [157]:
ytd_e_df = transform_historicals(ytd_df, elh_ytd_name)
ytd_e_df["%"] = ytd_e_df["%"] * 100
ytd_e_df = turn_to_percent(ytd_e_df, "%")

### Q2 and YTD data for PSEG Historical

In [158]:
q_p_df = transform_historicals(q_df, plh_q_name)
q_p_df["%"] = q_p_df["%"] * 100
q_p_df = turn_to_percent(q_p_df,"%")

In [159]:
ytd_p_df = transform_historicals(ytd_df,plh_ytd_name)
ytd_p_df["%"] = ytd_p_df["%"] * 100
ytd_p_df = turn_to_percent(ytd_p_df, "%")

### Q2 and YTD data for Dimension Historicals

In [160]:
q_combined_hist = [transform_historicals(q_df,name) for name in q_dim_names]

In [161]:
q_combined_hist = pd.concat(q_combined_hist, axis=0)
q_combined_hist["%"] = q_combined_hist["%"] * 100
q_combined_hist = turn_to_percent(q_combined_hist,"%")

In [162]:
ytd_combined_hist = [transform_historicals(ytd_df,name) for name in ytd_dim_names]

In [163]:
ytd_combined_hist = pd.concat(ytd_combined_hist,axis=0)
ytd_combined_hist["%"] = ytd_combined_hist["%"] * 100
ytd_combined_hist = turn_to_percent(ytd_combined_hist,"%")

# After completing all the transformations
We will save new sheets to the `transformed_data` folder

### Q2 Dataframes created for `transformed_data`
- Toplines
- KPI % All Brands
- PSEG KPIs
- Dimension KPIs %s & Rank
- Demographics KPI
- County KPI
- NPS
- PSEG Dimension Index Scores
- East Large Historical KPIs %s
- PSEG Historical KPIs %s Met Not-Met
- PSEG Dimension Historical %s

In [164]:
dfs_to_export = {
    'Toplines': q2_1,
    'KPI % All Brands': q2_kpi_df,
    'PSEG KPIs Met Not Met': q2_dim_kpi_df,
    'PSEG KPIs Difference': q2_dim_diff,
    'Dimension KPIs %s & Rank': q_kpi_r_all,
    'Demographics KPI': q_demo_df,
    'County KPI': q_c_df,
    'NPS': q_nps_df,
    'PSEG Dimension Index Scores': q_idx_df,
    'East Large Historical': q_e_df,
    'PSEG Historical': q_p_df,
    'Dimensions Historical': q_combined_hist
}

with pd.ExcelWriter(r'transformed_data\gas_residential_2025_q2.xlsx') as writer:
    for sheet_name, df in dfs_to_export.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)


### YTD Dataframes created for `transformed_data`
- Toplines
- KPI % All Brands
- PSEG KPIs
- Dimension KPIs %s & Rank
- Demographics KPI
- County KPI
- NPS
- PSEG Dimension Index Scores
- East Large Historical KPIs %s
- PSEG Historical KPIs %s Met Not-Met
- PSEG Dimension Historical %s

In [165]:
dfs_to_export = {
    'Toplines': ytd_1,
    'KPI % All Brands': ytd_kpi_df,
    'PSEG KPIs Met Not Met': ytd_dim_kpi_df,
    'PSEG KPIs Difference': ytd_dim_diff,
    'Dimension KPIs %s & Rank': ytd_kpi_r_all,
    'Demographics KPI': ytd_demo_df,
    'County': ytd_c_df,
    'NPS': ytd_nps_df,
    'PSEG Dimension Index Scores': ytd_idx_df,
    'East Large Historical': ytd_e_df,
    'PSEG Historical': ytd_p_df,
    'Dimensions Historical': ytd_combined_hist
}

with pd.ExcelWriter(r'transformed_data\gas_residential_2025_q2_ytd.xlsx') as writer:
    for sheet_name, df in dfs_to_export.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
